In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.ImageFolder('data/train', transform=transform)
test_data = datasets.ImageFolder('data/test', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=32, num_workers=2, pin_memory=True)

classes = train_data.classes

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3), # 48 -> 46
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3), # 46 -> 44
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2), # 44 -> 22

            nn.Dropout(0.25), # Early dropout

            nn.Conv2d(32, 64, kernel_size=3), # 22 -> 20
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3), # 20 -> 18
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2), # 18 -> 9

            nn.Conv2d(64, 128, kernel_size=3), # 9 -> 7
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2), # 7 -> 3

            nn.AdaptiveAvgPool2d((1, 1)) # output: (batch, 128, 1, 1)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(), # (batch, 128)
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(64, 7)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

def evaluate(model, dataloader):
    model.eval()
    total, correct, loss_total = 0, 0, 0.0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss_total += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return loss_total / len(dataloader), correct / total

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]", leave=False)
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        loop.set_postfix(loss=loss.item(), acc=correct/total)

    train_loss = running_loss / len(train_loader)
    train_acc = correct / total

    test_loss, test_acc = evaluate(model, test_loader)

    scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train loss: {train_loss:.4f} | Train acc: {train_acc:.2f}")
    print(f"Test loss: {test_loss:.4f} | Test acc: {test_acc:.2f}\n")

with open("classes.json", "w") as f:
    json.dump(classes, f)

torch.save(model.state_dict(), "model_cnn.pth")


Epoch 1/10
Train loss: 1.6421 | Train acc: 0.34
Test loss: 1.4603 | Test acc: 0.44



Epoch 2/10
Train loss: 1.4313 | Train acc: 0.45
Test loss: 1.3126 | Test acc: 0.50



Epoch 3/10
Train loss: 1.3393 | Train acc: 0.49
Test loss: 1.2330 | Test acc: 0.53



Epoch 4/10
Train loss: 1.2945 | Train acc: 0.51
Test loss: 1.2157 | Test acc: 0.54



Epoch 5/10
Train loss: 1.2600 | Train acc: 0.52
Test loss: 1.1882 | Test acc: 0.55



Epoch 6/10
Train loss: 1.2082 | Train acc: 0.55
Test loss: 1.1477 | Test acc: 0.57



Epoch 7/10
Train loss: 1.1871 | Train acc: 0.56
Test loss: 1.1426 | Test acc: 0.57



Epoch 8/10
Train loss: 1.1702 | Train acc: 0.56
Test loss: 1.1250 | Test acc: 0.58



Epoch 9/10
Train loss: 1.1665 | Train acc: 0.57
Test loss: 1.1045 | Test acc: 0.58



Epoch 10/10
Train loss: 1.1523 | Train acc: 0.57
Test loss: 1.1076 | Test acc: 0.58

